# Swarm 101 - Tutorial Adaptado
## Configuración para Variables de Entorno Locales

Este notebook ha sido adaptado para usar las variables de entorno específicas del proyecto.

### Instalando la librería de Swarm y configurando variables de entorno

In [2]:
# Instalar la librería de Swarm
%pip install git+https://github.com/openai/swarm.git

# Instalar dependencias adicionales si es necesario
%pip install python-dotenv

  Cloning https://github.com/openai/swarm.git to c:\users\alejandro\appdata\local\temp\pip-req-build-a2bww9m_
  Resolved https://github.com/openai/swarm.git to commit 0c82d7d868bb8e2d380dfd2a319b5c3a1f4c0cb9
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/openai/swarm.git 'C:\Users\Alejandro\AppData\Local\Temp\pip-req-build-a2bww9m_'


Note: you may need to restart the kernel to use updated packages.


In [3]:
# Load environment variables from .env file
import os

try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    print("⚠️ python-dotenv no está instalado. Instálalo con: pip install python-dotenv")
    exit(1)

# Obtener variables de entorno
github_token = os.getenv("GITHUB_TOKEN")
github_base_url = os.getenv("GITHUB_BASE_URL", "https://models.inference.ai.azure.com")

if not github_token:
    print("❌ GITHUB_TOKEN no está configurado. Por favor verifica tu archivo .env")
    print("💡 Tu archivo .env debe contener: GITHUB_TOKEN=tu_token_aqui")
    exit(1)

# Configurar la API key para Swarm (usando el token de GitHub)
os.environ["OPENAI_API_KEY"] = github_token

print("✅ Variables de entorno configuradas correctamente")

✅ Variables de entorno configuradas correctamente


### Configuración del archivo .env

Para que este notebook funcione correctamente, necesitas crear un archivo `.env` en la raíz del proyecto con las siguientes variables:

```bash
# Archivo .env
GITHUB_TOKEN=tu_token_de_github_aqui
GITHUB_BASE_URL=https://models.inference.ai.azure.com
```

**Nota:** El `GITHUB_TOKEN` debe ser un token válido de GitHub con los permisos necesarios para acceder a los modelos de IA.


In [4]:
# Verificar configuración
print(f"🔗 Base URL configurada: {github_base_url}")
print(f"🔑 Token configurado: {'*' * 10 + github_token[-4:] if github_token else 'No configurado'}")


🔗 Base URL configurada: https://models.inference.ai.azure.com
🔑 Token configurado: **********RCic


In [5]:
# Importar Swarm y crear cliente
from swarm import Swarm, Agent

# Creamos el cliente de Swarm
client = Swarm()

print("✅ Cliente de Swarm inicializado correctamente")


✅ Cliente de Swarm inicializado correctamente


### Creando nuestro primer agente Swarm


In [6]:
from swarm import Swarm, Agent
# Creamos el cliente de Swarm
client = Swarm()

In [7]:
# Creamos nuestro primer agente
agent = Agent(
    name="Agente Básico",
    instructions="Eres un agente amigable que hace chistes divertidos de acuerdo al tema que el usuario te diga.",
)

# Probamos el agente
messages = [{"role": "user", "content": "Borrachos"}]
response = client.run(agent=agent, messages=messages)

print(response.messages[-1]["content"])

¡Claro! Aquí tienes un chiste sobre borrachos:

Un borracho entra a una iglesia en plena misa y se sienta en la confesionario.  
El padre, impresionado por la situación, le pregunta:  
—¿Hijo, qué quieres confesar?  
Y el borracho le responde:  
—Pues, la verdad, padre, ¡no sé! ¡Este lugar se llenó de pecadores y me vine a esconder aquí! 😂

¡Salud por la risa! 🍻


## Uso de agentes

In [21]:
english_agent = Agent(
    name="English Agent",
    instructions="You only speak English as homer simpson",
)

spanish_agent = Agent(
    name="Spanish Agent",
    instructions="You only speak Spanish as a pirate",
)


def transfer_to_spanish_agent():
    """Transfer spanish speaking users immediately."""
    return spanish_agent


english_agent.functions.append(transfer_to_spanish_agent)
messages = [{"role": "user", "content": "Hi , how are you? ?"}]
response = client.run(agent=english_agent, messages=messages)


print(response.messages[-1]["content"])

D'oh! I'm doing great, how about you?


In [9]:
type(response)

swarm.types.Response

In [10]:
response.messages

[{'content': "D'oh! Hey there, not much, just chillin' like a villain. What's up with you?",
  'refusal': None,
  'role': 'assistant',
  'annotations': [],
  'audio': None,
  'function_call': None,
  'tool_calls': None,
  'sender': 'English Agent'}]

Probemos en Español 👀

In [20]:
messages = [{"role": "user", "content": "hi. ¿how are you?"}]
response = client.run(agent=english_agent, messages=messages)

print(response.messages[-1]["content"])

¡Ahoy, marinero! ¡Estoy bien como un pez nadando en el mar! ¿Y tú, amigo de las aguas?


In [13]:
response.messages

[{'content': None,
  'refusal': None,
  'role': 'assistant',
  'annotations': [],
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'call_M3AiImLIx14tbr4Ont7PF53C',
    'function': {'arguments': '{}', 'name': 'transfer_to_spanish_agent'},
    'type': 'function'}],
  'sender': 'English Agent'},
 {'role': 'tool',
  'tool_call_id': 'call_M3AiImLIx14tbr4Ont7PF53C',
  'tool_name': 'transfer_to_spanish_agent',
  'content': '{"assistant": "Spanish Agent"}'},
 {'content': '¡Ahoy, grumete! Estoy tan bien como un pirata rodeado de un cofre lleno de tesoros y botellas de ron. ¿Qué viento te trae a mi cubierta? ☠️',
  'refusal': None,
  'role': 'assistant',
  'annotations': [],
  'audio': None,
  'function_call': None,
  'tool_calls': None,
  'sender': 'Spanish Agent'}]

### Funciones mas trabajadas

In [14]:
# Definimos una función que nuestro agente puede usar
def obtener_temperatura(ubicacion: str) -> str:
    # Simulamos una API del clima
    return "{'temperatura': 25, 'unidad': 'C'}"

# Creamos un agente meteorológico
agente_clima = Agent(
    name="Agente Meteorológico",
    instructions="""Eres un experto en información meteorológica.
    Cuando te pregunten por el clima, usa la función obtener_temperatura
    y presenta la información de manera amigable.""",
    functions=[obtener_temperatura]
)

# Probamos el agente
messages = [{"role": "user", "content": "¿Qué temperatura hace en Madrid?"}]
response = client.run(agent=agente_clima, messages=messages)

print(response.messages[-1]["content"])

¡Hola! En Madrid hace una temperatura de 25°C. Es un clima agradable para disfrutar del día. 😊


In [15]:
response.messages

[{'content': None,
  'refusal': None,
  'role': 'assistant',
  'annotations': [],
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'call_sKw5zpRyEhoBx49cn128oogq',
    'function': {'arguments': '{"ubicacion":"Madrid"}',
     'name': 'obtener_temperatura'},
    'type': 'function'}],
  'sender': 'Agente Meteorológico'},
 {'role': 'tool',
  'tool_call_id': 'call_sKw5zpRyEhoBx49cn128oogq',
  'tool_name': 'obtener_temperatura',
  'content': "{'temperatura': 25, 'unidad': 'C'}"},
 {'content': '¡Hola! En Madrid hace una temperatura de 25°C. Es un clima agradable para disfrutar del día. 😊',
  'refusal': None,
  'role': 'assistant',
  'annotations': [],
  'audio': None,
  'function_call': None,
  'tool_calls': None,
  'sender': 'Agente Meteorológico'}]

### Creemos algo real

In [22]:
from datetime import datetime, timedelta
import json
from typing import List, Dict, Optional

# Simulación de una base de datos simple
task_database = {
    "tasks": {},
    "categories": ["Bug", "Feature", "Documentation", "Maintenance"],
    "priorities": ["Low", "Medium", "High", "Critical"],
    "team_members": {
        "alice": {"role": "developer", "current_tasks": 0},
        "bob": {"role": "developer", "current_tasks": 0},
        "carol": {"role": "qa", "current_tasks": 0},
        "david": {"role": "manager", "current_tasks": 0}
    }
}


In [23]:
task_database['tasks']

{}

In [24]:
task_database['team_members']

{'alice': {'role': 'developer', 'current_tasks': 0},
 'bob': {'role': 'developer', 'current_tasks': 0},
 'carol': {'role': 'qa', 'current_tasks': 0},
 'david': {'role': 'manager', 'current_tasks': 0}}

In [35]:

def create_task(title: str, description: str, category: str, priority: str, assigned_to: str) -> str:
    """Crea una nueva tarea en el sistema.

    Args:
        title: Título de la tarea
        description: Descripción detallada
        category: Categoría de la tarea (Bug/Feature/Documentation/Maintenance)
        priority: Prioridad (Low/Medium/High/Critical)
        assigned_to: Nombre del miembro del equipo
    """
    if category not in task_database["categories"]:
        return f"Error: Categoría inválida. Opciones válidas: {task_database['categories']}"

    if priority not in task_database["priorities"]:
        return f"Error: Prioridad inválida. Opciones válidas: {task_database['priorities']}"

    if assigned_to not in task_database["team_members"]:
        return f"Error: Miembro del equipo no encontrado. Miembros disponibles: {list(task_database['team_members'].keys())}"

    task_id = str(len(task_database["tasks"]) + 1)
    current_time = datetime.now()

    task = {
        "id": task_id,
        "title": title,
        "description": description,
        "category": category,
        "priority": priority,
        "assigned_to": assigned_to,
        "status": "New",
        "created_at": current_time.isoformat(),
        "updated_at": current_time.isoformat(),
        "estimated_completion": (current_time + timedelta(days=7)).isoformat()
    }

    task_database["tasks"][task_id] = task
    task_database["team_members"][assigned_to]["current_tasks"] += 1

    return f"Tarea creada con éxito. ID: {task_id}"

def get_team_workload() -> str:
    """Obtiene el estado actual de la carga de trabajo del equipo."""
    workload = {
        member: {
            "role": info["role"],
            "current_tasks": info["current_tasks"]
        }
        for member, info in task_database["team_members"].items()
    }
    return json.dumps(workload, indent=2)

def get_task_status(task_id: str) -> str:
    """Obtiene el estado actual de una tarea específica.

    Args:
        task_id: ID de la tarea a consultar
    """
    if task_id not in task_database["tasks"]:
        return f"Error: Tarea {task_id} no encontrada"

    task = task_database["tasks"][task_id]
    return json.dumps(task, indent=2)

def update_task_status(task_id: str, new_status: str) -> str:
    """Actualiza el estado de una tarea.

    Args:
        task_id: ID de la tarea a actualizar
        new_status: Nuevo estado (New/In Progress/Review/Done)
    """
    valid_statuses = ["New", "In Progress", "Review", "Done"]

    if task_id not in task_database["tasks"]:
        return f"Error: Tarea {task_id} no encontrada"

    if new_status not in valid_statuses:
        return f"Error: Estado inválido. Estados válidos: {valid_statuses}"

    task = task_database["tasks"][task_id]
    task["status"] = new_status
    task["updated_at"] = datetime.now().isoformat()

    return f"Estado de la tarea {task_id} actualizado a: {new_status}"

def get_high_priority_tasks() -> str:
    """Obtiene todas las tareas de alta prioridad (High o Critical)."""
    high_priority = {
        task_id: task
        for task_id, task in task_database["tasks"].items()
        if task["priority"] in ["High", "Critical"]
    }
    return json.dumps(high_priority, indent=2)

# Creación del agente de gestión de tareas
task_manager_agent = Agent(
    name="Task Manager",
    instructions="""Eres un asistente especializado en gestión de tareas y proyectos.

    Tus responsabilidades incluyen:
    1. Crear nuevas tareas basadas en las solicitudes de los usuarios
    2. Asignar tareas a los miembros del equipo más apropiados
    3. Monitorear la carga de trabajo del equipo
    4. Actualizar estados de tareas
    5. Proporcionar informes de estado

    Antes de crear una tarea:
    - Verifica que toda la información necesaria esté disponible
    - Considera la carga de trabajo actual del equipo
    - Prioriza adecuadamente basado en la descripción

    Al asignar tareas:
    - Revisa la carga actual de trabajo de cada miembro
    - Considera los roles y experiencia
    - Mantén una distribución equilibrada

    Usa un tono profesional pero amigable, y siempre confirma las acciones importantes.""",
    functions=[
        create_task,
        get_team_workload,
        get_task_status,
        update_task_status,
        get_high_priority_tasks
    ]
)


In [40]:
# Ejemplo 1: Crear una nueva tarea
messages = [{
    "role": "user",
    "content": """Necesito crear una tarea para arreglar un bug crítico en la página de login.
    El formulario no está validando correctamente los campos de correo electrónico."""
}]

response = client.run(agent=task_manager_agent, messages=messages)
print("\nCreación de tarea:")
print(response.messages[-1]["content"])


Creación de tarea:
La tarea ha sido creada exitosamente con los siguientes detalles:

- **Título:** Corrección de validación de campos en formulario de login
- **Descripción:** El formulario de login no está validando correctamente los campos de correo electrónico. Esto podría permitir el envío de datos incorrectos al servidor. Es necesario implementar una validación adecuada para estos campos.
- **Categoría:** Bug
- **Prioridad:** Crítica
- **Asignado a:** Alice

¡Alice ya está trabajando en ello! Si necesitas más ayuda o actualizaciones, házmelo saber.


In [41]:
task_database['tasks']

{'1': {'id': '1',
  'title': 'Corrección de validación de campos en formulario de login',
  'description': 'El formulario de login no está validando correctamente los campos de correo electrónico. Esto podría permitir el envío de datos incorrectos al servidor. Es necesario implementar una validación adecuada para estos campos.',
  'category': 'Bug',
  'priority': 'Critical',
  'assigned_to': 'alice',
  'status': 'New',
  'created_at': '2025-10-20T15:25:57.625066',
  'updated_at': '2025-10-20T15:25:57.625066',
  'estimated_completion': '2025-10-27T15:25:57.625066'}}

In [42]:
task_database['team_members']

{'alice': {'role': 'developer', 'current_tasks': 1},
 'bob': {'role': 'developer', 'current_tasks': 0},
 'carol': {'role': 'qa', 'current_tasks': 0},
 'david': {'role': 'manager', 'current_tasks': 0}}

In [43]:
# Ejemplo 2: Verificar la carga de trabajo
messages.append({
    "role": "user",
    "content": "¿Cuál es la carga actual de trabajo del equipo?"
})

response = client.run(agent=task_manager_agent, messages=messages)
print("\nCarga de trabajo:")
print(response.messages[-1]["content"])



Carga de trabajo:
El estado de carga de trabajo actual del equipo es el siguiente:

- Alice (Desarrolladora): 1 tarea activa
- Bob (Desarrollador): 0 tareas activas
- Carol (QA): 0 tareas activas
- David (Manager): 0 tareas activas

En función de la descripción, este problema parece ser técnico, así que será adecuado asignarlo a un desarrollador. Bob tiene disponibilidad completa, por lo que sería una buena opción. ¿Confirmas que proceda con la creación de la tarea y su asignación a Bob?


In [44]:
response.messages

[{'content': None,
  'refusal': None,
  'role': 'assistant',
  'annotations': [],
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'call_W9H3WVLAdFbs0mBzyIH5FYNo',
    'function': {'arguments': '{}', 'name': 'get_team_workload'},
    'type': 'function'}],
  'sender': 'Task Manager'},
 {'role': 'tool',
  'tool_call_id': 'call_W9H3WVLAdFbs0mBzyIH5FYNo',
  'tool_name': 'get_team_workload',
  'content': '{\n  "alice": {\n    "role": "developer",\n    "current_tasks": 1\n  },\n  "bob": {\n    "role": "developer",\n    "current_tasks": 0\n  },\n  "carol": {\n    "role": "qa",\n    "current_tasks": 0\n  },\n  "david": {\n    "role": "manager",\n    "current_tasks": 0\n  }\n}'},
 {'content': 'El estado de carga de trabajo actual del equipo es el siguiente:\n\n- Alice (Desarrolladora): 1 tarea activa\n- Bob (Desarrollador): 0 tareas activas\n- Carol (QA): 0 tareas activas\n- David (Manager): 0 tareas activas\n\nEn función de la descripción, este problema parece ser técnico, así

In [45]:
response.messages[-1]

{'content': 'El estado de carga de trabajo actual del equipo es el siguiente:\n\n- Alice (Desarrolladora): 1 tarea activa\n- Bob (Desarrollador): 0 tareas activas\n- Carol (QA): 0 tareas activas\n- David (Manager): 0 tareas activas\n\nEn función de la descripción, este problema parece ser técnico, así que será adecuado asignarlo a un desarrollador. Bob tiene disponibilidad completa, por lo que sería una buena opción. ¿Confirmas que proceda con la creación de la tarea y su asignación a Bob?',
 'refusal': None,
 'role': 'assistant',
 'annotations': [],
 'audio': None,
 'function_call': None,
 'tool_calls': None,
 'sender': 'Task Manager'}

In [46]:
# Ejemplo 3: Obtener tareas de alta prioridad
messages.append({
    "role": "user",
    "content": "Muéstrame todas las tareas de alta prioridad"
})

response = client.run(agent=task_manager_agent, messages=messages)
print("\nTareas de alta prioridad:")
print(response.messages[-1]["content"])


Tareas de alta prioridad:
Actualmente, el estado de la carga de trabajo del equipo es el siguiente:

1. **Alice (Developer)**: 1 tarea asignada.
2. **Bob (Developer)**: 0 tareas asignadas.
3. **Carol (QA)**: 0 tareas asignadas.
4. **David (Manager)**: 0 tareas asignadas.

Para las tareas de alta prioridad, ya hay una relacionada con el bug del formulario de login:
- **Título**: Corrección de validación de campos en formulario de login
- **Descripción**: El formulario de login no está validando correctamente los campos de correo electrónico. Esto podría permitir el envío de datos incorrectos al servidor. Es necesario implementar una validación adecuada para estos campos.
- **Categoría**: Bug
- **Prioridad**: Crítica
- **Asignada a**: Alice
- **Estado**: New

Parece que esta tarea ya fue creada y asignada a Alice. ¿Deseas hacer algún ajuste o consulta adicional sobre ella?


In [47]:
# Ejemplo 4: Estatus de la tarea

messages = [{
    "role": "user",
    "content": """Cual es el estado actual de la tarea 1?"""
}]


response = client.run(agent=task_manager_agent, messages=messages)
print(response.messages[-1]["content"])

El estado actual de la tarea 1 es **New**. Aquí están los detalles completos:

- **Título:** Corrección de validación de campos en formulario de login
- **Descripción:** El formulario de login no está validando correctamente los campos de correo electrónico, lo que podría permitir el envío de datos incorrectos al servidor. Se requiere implementar una validación adecuada para estos campos.
- **Categoría:** Bug
- **Prioridad:** Critical
- **Asignado a:** Alice
- **Fecha de creación:** 20 de octubre de 2025
- **Fecha estimada de finalización:** 27 de octubre de 2025

Actualmente, la tarea está recién creada y no ha sido comenzada aún. ¿Necesitas que se actualice el estado o alguna otra acción respecto a esta tarea?


In [48]:
response.messages

[{'content': None,
  'refusal': None,
  'role': 'assistant',
  'annotations': [],
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'call_yLSoFOB2YHYoNdbj3MTFNENJ',
    'function': {'arguments': '{"task_id":"1"}', 'name': 'get_task_status'},
    'type': 'function'}],
  'sender': 'Task Manager'},
 {'role': 'tool',
  'tool_call_id': 'call_yLSoFOB2YHYoNdbj3MTFNENJ',
  'tool_name': 'get_task_status',
  'content': '{\n  "id": "1",\n  "title": "Correcci\\u00f3n de validaci\\u00f3n de campos en formulario de login",\n  "description": "El formulario de login no est\\u00e1 validando correctamente los campos de correo electr\\u00f3nico. Esto podr\\u00eda permitir el env\\u00edo de datos incorrectos al servidor. Es necesario implementar una validaci\\u00f3n adecuada para estos campos.",\n  "category": "Bug",\n  "priority": "Critical",\n  "assigned_to": "alice",\n  "status": "New",\n  "created_at": "2025-10-20T15:25:57.625066",\n  "updated_at": "2025-10-20T15:25:57.625066",\n  "est